In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import chromedriver_autoinstaller
import time
from selenium.webdriver.common.by import By
import re
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pickle

In [20]:
def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--lang=ko_KR')
    chrome_options.add_argument('--window-size=1920x1080')

    # headless임을 숨기기 위해서
    # headless인 경우 Cloudflare 서비스가 동작한다.
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
    
    chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
    try:
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')   
    except:
        chromedriver_autoinstaller.install(True)
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')

    driver.implicitly_wait(10)
    
    return driver

In [34]:
# 드라이버 호출 -> 실행
driver = set_chrome_driver()

C:\Users\LSH\AppData\Local\Temp/ipykernel_1672/2544243862.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')


In [30]:
# base
driver.implicitly_wait(1)
driver.get('https://store.steampowered.com/')
time.sleep(1)

In [23]:
with open('review.pk' , 'rb') as f:
    eng_review = pickle.load(f)

In [24]:
game_name = eng_review['game_name'].tolist()

In [31]:
error = []
koreview_eval = []

In [26]:
def find_game(game, string):
    game = game.lower()
    string = string.lower()

    game_resub = re.sub('[^a-z0-9]','',game)
    string_resub = re.sub('[^a-z0-9I]','',string)
    
    if game_resub == string_resub:
        return True

In [38]:
not_in_list = ['people found this review', 'early access review', 'recommended', 'hrs on record'
                              , 'posted: ', 'products in account', 'product received for free', 
                              'person found this review funny', 'no one has rated this review as helpful yet']
num = 959 # 342
while num < 1001:
    driver.get('https://store.steampowered.com/')
    print('{}번째 크롤링'.format(num))
    try:
        name = game_name[num-1]
        print('{} 크롤링 중'.format(name))
        driver.find_element("xpath", '//*[@id="store_nav_search_term"]').send_keys(name)
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="store_search_link"]/img').click()
        time.sleep(1)

        for i in range(1,20):
            string = driver.find_element("xpath",'//*[@id="search_resultsRows"]/a[{}]/div[2]/div[1]/span'.format(i)).text
            if find_game(name, string):
                driver.find_element("xpath",'//*[@id="search_resultsRows"]/a[{}]/div[2]/div[1]/span'.format(i)).click()
                break
        time.sleep(1)
    except:
        name = re.sub("[^A-Za-z0-9']+",' ',name)
        print('{} 크롤링 중'.format(name))
        driver.find_element("xpath", '//*[@id="store_nav_search_term"]').send_keys(name)
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="store_search_link"]/img').click()
        time.sleep(1)

        for i in range(1,20):
            string = driver.find_element("xpath",'//*[@id="search_resultsRows"]/a[{}]/div[2]/div[1]/span'.format(i)).text
            if find_game(name, string):
                driver.find_element("xpath",'//*[@id="search_resultsRows"]/a[{}]/div[2]/div[1]/span'.format(i)).click()
                break
        time.sleep(1)
    

    
    # 19+
    try:
        driver.find_element("xpath", '//*[@id="app_agegate"]/div[1]/div[3]')
        driver.find_element("xpath", '//*[@id="ageYear"]').send_keys('1997')
        driver.find_element(By.XPATH, '//*[@id="view_product_page_btn"]').click()
        time.sleep(1)
    except Exception as e:
        print(e)
        
    # get review
    # review = [ { review : good / not good }, ... ]
    # order by 'helpful' desc
    # if review count >= 100: stop
    review_count = 0
    # scroll event
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
    try:
        # click all review
        driver.find_element(By.XPATH, '//*[@id="ViewAllReviewssummary"]/a').click()
        time.sleep(2)
    
        # community error
        try:
            driver.find_element("xpath", '//*[@id="ViewAllForApp"]').click()
            driver.find_element("xpath", '//*[@id="age_gate_btn_continue"]/span').click()
            time.sleep(1)
        except Exception as e:
            print(e)

        # choose korean
        driver.find_element("xpath", '//*[@id="filterlanguage"]/div[2]').click()
        time.sleep(1)
        driver.find_element("xpath", '//*[@id="filterlanguage_option_5"]'.format(i)).click()
        time.sleep(1)
        #for i in range(1,10):
        #    lang = driver.find_element("xpath", '//*[@id="filterlanguage_option_{}"]'.format(i)).text
        #    lang = lang.lower()
        #    if lang == 'korean':
        #        driver.find_element("xpath", '//*[@id="filterlanguage_option_{}"]'.format(i)).click()
        #        break
        
        dict_review = {}
        try:
            # page cycle
            for page in range(1, 100):
    
                for _ in range(2):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)

                review_page = driver.find_element("xpath", '//*[@id="page{}"]'.format(page)).text
                pages = review_page.split('\n')

                evaluated = []
                for senten in pages:
                    sen = senten.lower()
                    if sen == 'recommended':
                        evaluated.append('1')
                    if sen == 'not recommended':
                        evaluated.append('0')


                review_lists = []
                for senten in pages:
                    sen = senten.lower()
                    if 'people found this review' not in sen:
                        if 'early access review' not in sen:
                            if 'recommended' not in sen:
                                if 'hrs on record' not in sen:
                                    if 'posted: ' not in sen:
                                        if 'products in account' not in sen:
                                            if 'product received for free' not in sen:
                                                if 'person found this review funny' not in sen:
                                                    if 'no one has rated this review as helpful yet' not in sen:
                                                        review_lists.append(sen)

                preprocessed_reviews = []
                review_split = []

                for sen in review_lists:
                    if sen.isdigit():
                        review_split = " ".join(review_split)
                        preprocessed_reviews.append(review_split)
                        review_split = []
                    else:
                        review_split.append(sen)

                for sen, evaluate in zip(preprocessed_reviews, evaluated):
                    dict_review[sen] = int(evaluate)

        except:
            print('Page_rangeout')
            pass
    
        koreview_eval.append(dict_review)
    except:
        print('no review')
        koreview_eval.append('No review')
    
    if num % 50 == 0:
        with open('2koreview_eval{}.pk'.format(num) , 'wb') as f:
            pickle.dump(koreview_eval, f)
            

    time.sleep(1)

    num +=1

959번째 크롤링
Fallen girl - Black rose and the fire of desire 크롤링 중
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app_agegate"]/div[1]/div[3]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Page_rangeou

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ViewAllForApp"]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Page_rangeout
965번째 크롤링
Warhammer: Chaosbane 크롤링 중
Message: no such element: Unable to l

Page_rangeout
970번째 크롤링
DARK SOULS™ II 크롤링 중
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app_agegate"]/div[1]/div[3]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Message: no such element: Unabl

Page_rangeout
975번째 크롤링
Gunvolt Chronicles: Luminous Avenger iX 크롤링 중
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app_agegate"]/div[1]/div[3]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Messag

Page_rangeout
980번째 크롤링
Onimusha: Warlords 크롤링 중
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app_agegate"]/div[1]/div[3]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Page_rangeout
981번째 크롤링
Sai

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ViewAllForApp"]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Page_rangeout
986번째 크롤링
The Ramen Sensei 크롤링 중
Message: no such element: Unable to locat

Page_rangeout
990번째 크롤링
Counter Fight: Samurai Edition 크롤링 중
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app_agegate"]/div[1]/div[3]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Message: no suc

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ViewAllForApp"]"}
  (Session info: chrome=105.0.5195.128)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062C0A3+2212003]
	Ordinal0 [0x005C2CC1+1780929]
	Ordinal0 [0x004D465D+804445]
	Ordinal0 [0x00503475+996469]
	Ordinal0 [0x0050363B+996923]
	Ordinal0 [0x00531382+1184642]
	Ordinal0 [0x0051EC64+1109092]
	Ordinal0 [0x0052F5B2+1177010]
	Ordinal0 [0x0051EA36+1108534]
	Ordinal0 [0x004F83C9+951241]
	Ordinal0 [0x004F9396+955286]
	GetHandleVerifier [0x008D9CE2+2746722]
	GetHandleVerifier [0x008CA234+2682548]
	GetHandleVerifier [0x006BB34A+524234]
	GetHandleVerifier [0x006B9B60+518112]
	Ordinal0 [0x005C9FBC+1810364]
	Ordinal0 [0x005CEA28+1829416]
	Ordinal0 [0x005CEB15+1829653]
	Ordinal0 [0x005D8744+1869636]
	BaseThreadInitThunk [0x75C76739+25]
	RtlGetFullPathName_UEx [0x77358FD2+1218]
	RtlGetFullPathName_UEx [0x77358F9D+1165]

Page_rangeout
996번째 크롤링
Close Combat: The Bloody First 크롤링 중
Message: no such element: U

Page_rangeout


In [28]:
koreview_eval

[{'재밌는 게임이 핵쟁이 새끼들 때문에 최단기 퇴물이라는 비운의 타이틀을 얻게 된 게임입니다. 에이씻팔 레전드 만들지 말고 스팀에서도 핵 쓰는 새끼들은 나가 뒤지시기 바랍니다.  무엇보다 스팀에서 실행하면 썩어빠진 오리진 런처 이중 실행을 안 해도 오리진 계정과 연동 되어서 게임 실행이 된다는 게 최고의 장점이 아닐까 싶습니다.   neokgu': 1,
  '라이브러리 - [] 우클릭 [] - [] - "-dev" 입력하면 게임 시작시 ea 소리 사라집니다   낙서': 1,
  '서버 관리 안하냐?   #waldo': 0,
  '스킨 디자이너들 싹 다 자르고 새로 뽑아라   skjfksd': 1,
  '겜 킬때 ea뜨는 화면 없애는 법을 찾고 광명을 찾앗습니다 감사합니다   꿀벌은우리의친구': 1,
  '친구랑 하지 마세요 백민서 씨발새끼   백민서 따까리': 1,
  '공룡나옴   ( ˘▽˘)': 1,
  '여러분이 스팀으로 에이펙스 레전드를 해야하는 이유 간단합니다  오리진을 안켜도 됩니다. ea가 뭘 잘못 처먹었는지 자사 클라를 안띄우고 자연스럽게 스팀으로도 킬수있게 만들어줬어요 이건 대단합니다! 왜냐고여? 에픽게임즈 가보셨습니까? ea게임 하나사면 어디서 키든 에픽,오리진 둘다켜야되요 아니 그럼 오리진만 키고 이용하면 되는거 아니냐? 에픽에서 사면 오리진으로 실행하고 에픽클라를 한번 더 띄워서 검증합니다. 스팀에서 유비게임 사서 유비클라로 실행했다고 유비가 스팀클라 한번 더 띄워서 확인절차합니까? 근데 ea는 그 ㅈㄹ을 합니다. 아주 이상한 새끼들이죠 근데 여기서는 묻지도 따지지도 않고 프리패스라 이 말입니다.  거기에 자사에서 파는 에디션까지 전부 스팀출시! 이럴수가?! 매번 돌아오는 스팀 세일적용까지 해줍니다. 아주 미쳐도 단단히 미쳤죠 에디션 지금 아무거나 켜보세요 거기안에는 100%확율료 600~1000캐쉬가 들어있습니다 이게 얼마냐? 놀라지 마세요 인게임으로 11101입니다 10은 뭐냐? ea는 아주 꽉꽉막힌 미친새끼들이라 달러환율을 일원짜리

In [47]:
    with open('new_koreview_eval.pk'.format(num) , 'wb') as f:
        pickle.dump(koreview_eval, f)

In [46]:
len(koreview_eval)

1000

In [53]:
new = koreview_eval

In [56]:
for 

1000

In [57]:
for dic in new:
    if dic == 'No review':
        print(dic)
        
        

No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review
No review


In [50]:
for dic in koreview_eval:
    for (key, value) in dic.items():
        copy_review = key.split('  ')
        print(copy_review[-1])

 neokgu
 낙서
 #waldo
 skjfksd
 꿀벌은우리의친구
 백민서 따까리
 ( ˘▽˘)
 mr.bonggoo
 이해창
 3sik
 chickenkun
 moclo
현
 mr.bread
 kimsi
 mandoo
 양재민입니다
 개인의실책,개별의선택
 01
 꽃피는봄이오면
 노훈
 jaejin2005
 huniadventure
 jack the ripper
 ㅁ도트세상ㅁ
 빅비
 뭉라
 meiblue2(입대)
 overdose
 souffle
 leemg
 call an ambulance but not for me
 2시저그
 집문서와맞바꾼실드배터리
 resbik
 hnr
 hungbok
 tfjumper
 차차0412
 reppey
 i3eancurd
 superbgimchi
 murang
 paul
 crayon
 sun_in_jang
 freeslok
 black rabbit
 pool_jin
 sunfish
 chotroll
 astella
 jun__man
 라기봉
 pettyriot
 이나즈마성군
 sadcat
 shock&awe
 流進eugene
 ta
 자주색
 3l
 xcat0303
 mtp_danhana
 teamcat
 noway
 [rad - zangoose]
 포우
 차해인
 dantodanto
 locojung
 sna_renegades
 trololo
 킴 솔
 off
 정석형
 sona
 overdeu
 ssap_
 aaaaaaaaaa
 예준
 os
 roy0213
 ユジン部長
 mikuaaa
 ddno
 =dh$=nick
 baron
 mingjigi
 tansa
 eunji
 와요2341
 blackhistory
 tamdaa
 htw4557
 songhyeon
 buple
 yangmomill
 freedom
 bakyasi01
 nichipe
 홍
 l'usine
 cherry
kanaya
 youngbba
 hobat
 나냐
 yumammu boomu
 bzrwlr
 sth.merits
 김초희
 o'-'o
 에

 seoback
 무면허
 츄지
 레옹
 halo
 wanderroo
 ddong _d
 스작
 woop
 hanas
 배가고파
 tuljoomuck
 ehdgus5199
 sakshin
 ljy1663
 headshop
 blue
 yourfa
 momotoren
 sw jeong
 justgentle...
 rex
 fksan78
 yamu1005
 아코
 ywnhk-xλ˥∀
 원래버섯이빨리자라냐
 foxlimms
 hedkandi
 푸렙
 hotgamja
 ashgray im
 sak
 lukas
 lord shin
 stun
 haru-man(하루맨)
 르콤트
 손우빈
 perytails
 serena
 abin
 vmfls
 v4eyes
 hart
 cho_jin
 우후훗
 seonghormone
 goodluck
 이천쌀
 kimchi warrior 1 product in account
 닭
 dogma
 winter
 foxhound
 melt-eachother
 약드시는 여왕님♥
 master_baiting
 hodu8051
 포아이
 flatline_wp
 탕탕충
 stkang
 cafelatte
 nomfeng
 rainbowpart
 djinping
 hyungmuk89
 dgpistol
 ddt
 pyh
 reppey
 kimdeokja
 robotic
 박가놈
 zolapsy
 daehyup1124
 kijinll
 mint
 djinn
 kimchis never wait
 undead paladin
 danielthelee
 wwwwww
 hahn.dm
 puhaha380
 sungeumo
 windseeker
 bluesdrive
 balentine
 the doctor
 [kor] hong-gil-dong🔞
 daisyp
 marinewave
 sun_in_jang
 bravosix
 gyulmung
 um_bong
 오록청
 dobby
 jazzcursaders
 죠달호
 chungseok2
 arkous(아르쿠스)
 bok5ki

 겨울나무
 까망달
 이준싱
 column
 leena
 뽐돌이
 sı
 black rosemary
 lafiel
 zooarang
 춘식이
 meuhokk
 anowi
 tpof7777
 emperte
 dkzz
 ink
 reol- 煽げや尊し /第六感
 초요
 parksicheol
 akasopu
 scan
 rehall
 쉬이이버얼
 ㅇㅈㅇ
 meronot
 kts0814
 carrotforce
 민트핫초코
 ova
 해녀왕
 bäzyrak_karlcoux
 ajingang
 silan
 이루칸
 plz don't kill me
 xtrakun
 xemo
 rightcome
 dryingcow
 hragom
 자주색
 흑일귀
 reppey
 yesno
 everwater
 baboall
 maxbong
 dragonic21
 깐
 mystic
 ❖esill
 banshee
 혜지는혜지
 만두
 readcat
 ranger_squirrel
 thalia
 gordon ramsay
 kim writer
 이준호
 yhk
 70krw
hitmeonly
 paple
 bmoon
 kisozz1
 표윤찬
 rasputin
 serlise
 どらん
 노바코
 simo
 malseok
 mercy
 liotail
 samwol*
 han
 aekop
 무직백수
 hola
 문과
 goguma
 juicycool
 rudehd
 medoctor
 adebird (적당새)
 모닝마끼아또
 adfalcon
 kaz
 李榮浩
 niloc
 man_of_kimchi
 gurumi
 < blank >
 라브소소
 mari
 stranger
 코끼리쭈뿌쭈뿌
 a:dun
 beremo
 taeyoon
 seagull
 bigfish
 5월 24일 입대
 santafe
 눈빛날개
 hecto pascal kick
 b-shiba
 halfdrygrape
 j
 발푸딩
 alive coconut
 日泳
 ddangsik
 보노
 jester
 .
 여성당비례대표박종우
 mriy@
 u

 s2e3t7
 n4rcolepsy
 foxmagician
 masterkey
 때구르르
 하핫
 llow2
 hotfoxy
 rafaelo
 호갱
 express999
 why my wingman is so broken?
 west taiwan
 폭주_고양이
 손무
 팜천복
 hello
 홀리바이불
 jjodabeto
 hera
 망원동 지방 호족
 kimgodong
 kuhellhell
 aallaass29
 assov
 dagoku
 jys116
 jivi14
 driftingsoul
 wjdghkschl25
 cheminu
 junseong26
 dlaguswo11
 capitalism ho!
 红卫兵
 ulringring
 myaro
 aoe-410 ash october
 범그루말
 redbeanbread
 aznable2
 qt5123
 묘소향
 dubhe_lee
 ravenll
 골든우라늄
 sang ho shin
 arukei
 welsper
 dong
 gyongha12
 와웅
 yagun
 jackfrost
 ctan124
 ranteria
 deathscythek
 bupul
 dongjjang
 jipungtwe
 gks7563
 isodd
 uz
 samuel
 fox
 rename
 darkmetar
 bluesmile
 盧の意志を継ぐ者
 stkdhp
 some one
 idiothead
 orpoja
 nya nya
 harbinger
 paranbicycle
 rkdsm5
 ssalman
 nightmare knight
 tomatopope
 asukara
 archem
 region1390
 튼튼한노예
 red3543
 bwhiteshark
 [한] seuck
 dlrgh2698
 snipegod101
 lion
 twha98
 moe
 boxboy
 aleionx
 sheep duck
 nayu
 ඞ
 lemoncake
 sin_se_gy
 star2068
 들깨칼국수
 나비나무
 mcthewawa
 redeye
 노르뭬미숲고먐

 monika.chr
 hc
 윤월
 sc_reamer
 hsy123455
 name20
 다아알이
 seaskiming
 krhammer
 bjh1179156
 트 윗
 doll
 hrt
 adduaduddi_
 marong♥
 moshi
 2haedo
 はく ひかり
 루냥🦴
 whitsuntide
 hisora0822
 i_n_yong
 중국의제왕진핑시
 【🍵】
 하루살이
 luhata
 hitky
 dahyun❤
 evar
 ruptung
 minyon
 janyang
 manyong
 milkis
 n4vn
 도래미
 handle4456
 papipa
 렌짱w
 행개
 hanwoo
 123
 realbk
 rammeobu.
 pseul
eimminam
 현자타임
 sword man
 sonorous0528
 tnwprkawkclq
 dobonge
 xyz
 me
 gongchani
 yellow
you know how i feel?
 churi
 adrian schmitt
 justearth
 cm0312
 sldoal
 guri
 조평신
 god.5.0
 butterapple
 tibe
 senju
 avorzen
 jungjkch03
 ❀ʚ v 1 $ t a ¥ ɞ❀ ت
 안산에선 안산다
 badaimscripter
 [sukhamokdo]
 caffeinethief
 great_dog
 double j.
 favori
 꾸까
 kudam
 solo kim sejeong sesang
 앰끼리
 여고생쟝
 moonbom
 parkinsonruli
 김민성
 molm(몰름)
 h_seo
 pinkzebra
 cyber
 i succs eng
 dye0212
 staying_light
 rxonlyy3
 m_2
 angrybeastman
 ❈newworld
 emmintchoco
 빠앙통조림
 anixs
 issac
 charizard
 but i love you
 금은링
 shikinyang
 saxon
 룰랑이
 egg
 zicov
 syoukin
 

AttributeError: 'str' object has no attribute 'items'

In [ ]:
for (key, value) in review['review'][num].items():

In [77]:
with open('review.pk', 'rb') as f:
    review = pickle.load(f)

In [79]:
review['koreview'] = koreview_eval

In [89]:
review['koreview'][0]

{'재밌는 게임이 핵쟁이 새끼들 때문에 최단기 퇴물이라는 비운의 타이틀을 얻게 된 게임입니다. 에이씻팔 레전드 만들지 말고 스팀에서도 핵 쓰는 새끼들은 나가 뒤지시기 바랍니다.  무엇보다 스팀에서 실행하면 썩어빠진 오리진 런처 이중 실행을 안 해도 오리진 계정과 연동 되어서 게임 실행이 된다는 게 최고의 장점이 아닐까 싶습니다.': 1,
 '라이브러리 - [] 우클릭 [] - [] - "-dev" 입력하면 게임 시작시 ea 소리 사라집니다': 1,
 '서버 관리 안하냐?': 0,
 '스킨 디자이너들 싹 다 자르고 새로 뽑아라': 1,
 '겜 킬때 ea뜨는 화면 없애는 법을 찾고 광명을 찾앗습니다 감사합니다': 1,
 '친구랑 하지 마세요 백민서 씨발새끼': 1,
 '공룡나옴': 1,
 '여러분이 스팀으로 에이펙스 레전드를 해야하는 이유 간단합니다  오리진을 안켜도 됩니다. ea가 뭘 잘못 처먹었는지 자사 클라를 안띄우고 자연스럽게 스팀으로도 킬수있게 만들어줬어요 이건 대단합니다! 왜냐고여? 에픽게임즈 가보셨습니까? ea게임 하나사면 어디서 키든 에픽,오리진 둘다켜야되요 아니 그럼 오리진만 키고 이용하면 되는거 아니냐? 에픽에서 사면 오리진으로 실행하고 에픽클라를 한번 더 띄워서 검증합니다. 스팀에서 유비게임 사서 유비클라로 실행했다고 유비가 스팀클라 한번 더 띄워서 확인절차합니까? 근데 ea는 그 ㅈㄹ을 합니다. 아주 이상한 새끼들이죠 근데 여기서는 묻지도 따지지도 않고 프리패스라 이 말입니다.  거기에 자사에서 파는 에디션까지 전부 스팀출시! 이럴수가?! 매번 돌아오는 스팀 세일적용까지 해줍니다. 아주 미쳐도 단단히 미쳤죠 에디션 지금 아무거나 켜보세요 거기안에는 100%확율료 600~1000캐쉬가 들어있습니다 이게 얼마냐? 놀라지 마세요 인게임으로 11101입니다 10은 뭐냐? ea는 아주 꽉꽉막힌 미친새끼들이라 달러환율을 일원짜리까지 적용해놨어요 아마 소수점까지 적용 못한게 얘네한테는 천추의 한일겁니다. ea는 그런새끼들이란 말입니다 여러분 근데 그걸 세

In [94]:
game_names = review['game_name'].tolist()

In [95]:
en_review = pd.DataFrame()

In [96]:
en_review['game'] = game_names

In [97]:
en_review

,game
0,Apex Legends™
1,PUBG: BATTLEGROUNDS
2,MONSTER HUNTER RISE
3,Days Gone
4,Soulworker : Your Destiny Awaits
...,...
995,Close Combat: The Bloody First
996,Roboquest
997,PixARK
998,Hell Let Loose


In [111]:
en_review_apex = pd.DataFrame()

In [117]:
game_names[0]

'Apex Legends™'

In [114]:
en_review_apex['review'] = review_key

In [115]:
en_review_apex['evaluation'] = eval_value

In [119]:
en_review_apex = en_review_apex.fillna(game_names[0])
en_review_apex

,game,review,evaluation
0,Apex Legends™,at least you don't have to build an apartment ...,1
1,Apex Legends™,finally here it launches without origin... th...,1
2,Apex Legends™,finally i can remove origin.,1
3,Apex Legends™,uninstall origin,1
4,Apex Legends™,no need for origin? here is my positive review!,1
...,...,...,...
947,Apex Legends™,1 person found this review funny 0 product rec...,1
948,Apex Legends™,cured my depression with loba's ass,1
949,Apex Legends™,"apex is a cool game, imo its the best battle r...",1
950,Apex Legends™,"400+hours spent on the origin version, 670 on ...",1


In [185]:
def review_divide(review, num, all_review):
    game = review['game_name'][num]
    review_key = []
    eval_value = []
    for (key, value) in review['review'][num].items():
        if '1 person found this review funny' in key:
            key = key.replace('1 person found this review funny', '')
        if 'no one has rated this review as helpful yet':
            key = key.replace('no one has rated this review as helpful yet','')
        if '1 person found this review helpful' in key:
            key = key.replace('1 person found this review helpful','')
        if 'product received for free' in key:
            key = key.replace('product received for free','')
        key = ' '.join(key.split())
        if key == '':
            key = 'No review'
        if key[0].isdigit():
            key = key.replace(key[0],'',1)
        review_key.append(key)
        eval_value.append(value)
    game_review = pd.DataFrame()
    game_review['game'] = game
    game_review['review'] = review_key
    game_review['evaluation'] = eval_value
    game_review = game_review.fillna(game)
    all_review = pd.concat([all_review,game_review], ignore_index=True)
    return all_review

In [187]:
all_review = pd.DataFrame({'game':[], 'review':[], 'evaluation':[]})

In [188]:
for num in tqdm(range(1000)):
    all_review = review_divide(review, num, all_review)

100%|██████████| 1000/1000 [00:21<00:00, 46.62it/s]


In [189]:
all_review

,game,review,evaluation
0,Apex Legends™,at least you don't have to build an apartment ...,1.0
1,Apex Legends™,finally here it launches without origin... tha...,1.0
2,Apex Legends™,finally i can remove origin.,1.0
3,Apex Legends™,uninstall origin,1.0
4,Apex Legends™,no need for origin? here is my positive review!,1.0
...,...,...,...
695058,CARRION,i am taking you to brazil,1.0
695059,CARRION,being a big ol monster is a nice change of pa...,1.0
695060,CARRION,carrion is a fun and interesting little actio...,1.0
695061,CARRION,this game is exactly what it says it is- you ...,1.0


In [190]:
all_review["evaluation"] = all_review["evaluation"].astype(int)

In [214]:
del_index = all_review.loc[all_review['review'] == 'No review'].index

In [216]:
all_review = all_review.drop(index = del_index)

In [217]:
with open('english_review', 'wb') as f:
    pickle.dump(all_review, f)

In [206]:
def ko_review_divide(review, num, all_review):
    game = review['game_name'][num]
    review_key = []
    eval_value = []
    if review['koreview'][num] == 'No review':
        game_review = pd.DataFrame()
        game_review['game'] = game
        game_review['review'] = 'No reivew'
        game_review['evaluation'] = 'No reivew'
        game_review = game_review.fillna(game)
        all_review = pd.concat([all_review,game_review], ignore_index=True)
        return all_review
    for (key, value) in review['koreview'][num].items():
        if '1 person found this review funny' in key:
            key = key.replace('1 person found this review funny', '')
        if 'no one has rated this review as helpful yet':
            key = key.replace('no one has rated this review as helpful yet','')
        if '1 person found this review helpful' in key:
            key = key.replace('1 person found this review helpful','')
        if 'product received for free' in key:
            key = key.replace('product received for free','')
        key = ' '.join(key.split())
        if key == '':
            key = 'No review'
        if key[0].isdigit():
            key = key.replace(key[0],'',1)
        review_key.append(key)
        eval_value.append(value)

    game_review = pd.DataFrame()
    game_review['game'] = game
    game_review['review'] = review_key
    game_review['evaluation'] = eval_value
    game_review = game_review.fillna(game)
    all_review = pd.concat([all_review,game_review], ignore_index=True)
    return all_review

In [203]:
all_ko_review = pd.DataFrame({'game':[], 'review':[], 'evaluation':[]})

In [207]:
for num in tqdm(range(1000)):
    all_ko_review = ko_review_divide(review, num, all_ko_review)

100%|██████████| 1000/1000 [00:11<00:00, 87.86it/s]


In [208]:
all_ko_review[]

,game,review,evaluation
0,Apex Legends™,재밌는 게임이 핵쟁이 새끼들 때문에 최단기 퇴물이라는 비운의 타이틀을 얻게 된 게임...,1.0
1,Apex Legends™,"라이브러리 - [] 우클릭 [] - [] - ""-dev"" 입력하면 게임 시작시 ea...",1.0
2,Apex Legends™,서버 관리 안하냐?,0.0
3,Apex Legends™,스킨 디자이너들 싹 다 자르고 새로 뽑아라,1.0
4,Apex Legends™,겜 킬때 ea뜨는 화면 없애는 법을 찾고 광명을 찾앗습니다 감사합니다,1.0
...,...,...,...
366265,CARRION,한 번이라도 촉수괴물이 되는 자신을 상상해보셨다면 이 게임이 정답입니다. xd,1.0
366266,CARRION,아이고 힘들다,1.0
366267,CARRION,퍼즐도 그렇게 어렵지않고 난이도도 엄청 어려운 게임은 아닙니다. 다만 네비 시스템...,1.0
366268,CARRION,엑스포때 딱보고 컨셉이 너무 맘에 들어서 그대로 질렀는데 그만큼 너무 재밌었던거같...,1.0


In [218]:
del_index = all_ko_review.loc[all_ko_review['review']=='No review'].index
all_ko_review = all_ko_review.drop(index = del_index)

In [221]:
all_ko_review["evaluation"] = all_ko_review["evaluation"].astype(int)

In [222]:
all_ko_review

,game,review,evaluation
0,Apex Legends™,재밌는 게임이 핵쟁이 새끼들 때문에 최단기 퇴물이라는 비운의 타이틀을 얻게 된 게임...,1
1,Apex Legends™,"라이브러리 - [] 우클릭 [] - [] - ""-dev"" 입력하면 게임 시작시 ea...",1
2,Apex Legends™,서버 관리 안하냐?,0
3,Apex Legends™,스킨 디자이너들 싹 다 자르고 새로 뽑아라,1
4,Apex Legends™,겜 킬때 ea뜨는 화면 없애는 법을 찾고 광명을 찾앗습니다 감사합니다,1
...,...,...,...
366265,CARRION,한 번이라도 촉수괴물이 되는 자신을 상상해보셨다면 이 게임이 정답입니다. xd,1
366266,CARRION,아이고 힘들다,1
366267,CARRION,퍼즐도 그렇게 어렵지않고 난이도도 엄청 어려운 게임은 아닙니다. 다만 네비 시스템...,1
366268,CARRION,엑스포때 딱보고 컨셉이 너무 맘에 들어서 그대로 질렀는데 그만큼 너무 재밌었던거같...,1


In [ ]:
ddf.reset_index(drop=True)

In [223]:
with open('korea_review', 'wb') as f:
    pickle.dump(all_ko_review, f)